In [ ]:
import os
import pandas as pd
import cv2
import numpy as np 
import pandas as pd 
import time
import matplotlib.pyplot as plt
from skimage import measure
import matplotlib.patches as patches
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# from sklearn.utils import resample
# from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import resample
from keras.preprocessing.image import ImageDataGenerator
#imports
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten,Conv2D,GlobalMaxPooling2D
from keras import optimizers
from keras.preprocessing import image
from keras.metrics import top_k_categorical_accuracy
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import Adam
from keras.models import model_from_json, load_model
import time

print(os.listdir("../input"))
print(os.listdir("../input/humpback-whale-identification"))
print(os.listdir("../input/pretrained-model"))
# Any results you write to the current directory are saved as output.

In [ ]:
DATA="../input/humpback-whale-identification"
TRAIN_IMG="../input/humpback-whale-identification/train"
TEST_IMG="../input/humpback-whale-identification/test"
PRETRAIN_MODEL="../input/pretrained-model/inceptionresnetv2_model_bb_last.h5"
PRETRAIN_WEIGHTS="../input/pretrained-model/weights_inceptionresnetv2.hdf5"

BB_DATA="../input/bounding-boxes-using-image-processing"

In [ ]:
LOAD_MODEL=True

In [ ]:
test_df= pd.DataFrame({"Image":  os.listdir(TEST_IMG)})
print("test images:"+ str(len(test_df)))

In [ ]:
train_lbl = pd.read_csv(os.path.join(DATA, 'train.csv'))
print("train images:"+ str(len(train_lbl)))
print("total unique class:"+ str(len(np.unique(train_lbl['Id']))))
train_lbl.head()

**Balance training set**

In [ ]:
train_lbl['newwhale_lbl']=np.where(train_lbl['Id']=='new_whale',1,0)

In [ ]:
#take out whales with less than 2 examples
df=train_lbl.groupby(['Id']).size().reset_index(
    name='train_examples')
df=df[df['train_examples']>=2]

print("number of classes with more than 2 examples:"+ str(len(df)))
# train_lbl_balance=train_lbl[train_lbl['Id'].isin(df['Id'])]
# print("number of train instances :"+ str(len(train_lbl_balance)))

In [ ]:
#remove whales with a single train example (2072 examples)
train_lbl=train_lbl[train_lbl['Id'].isin(df['Id'])]
print("number of train instances after removal:"+ str(len(train_lbl)))


In [ ]:
#remove problematic image
train_lbl=train_lbl[train_lbl['Image']!='859e1399e.jpg']
print("number of train instances after removal:"+ str(len(train_lbl)))

In [ ]:
# #remove new_whale
# train_lbl=train_lbl[train_lbl['newwhale_lbl']==0]
# print("number of train instances after removal:"+ str(len(train_lbl)))

# Bounding boxes

In [ ]:
#read BB data fo train and test
train_bb=pd.read_csv(os.path.join(BB_DATA,"boxs_train.csv"))
test_bb=pd.read_csv(os.path.join(BB_DATA,"boxs_test.csv"))
bb_all=train_bb.append(test_bb, ignore_index=True)

bb = {}
for i in range(len(bb_all)):
    image=bb_all['Image'].iloc[i].split("/")[-1]
    x0=float(bb_all['x0'].iloc[i])
    y0=float(bb_all['y0'].iloc[i])
    x1=float(bb_all['x1'].iloc[i])
    y1=float(bb_all['y1'].iloc[i])
    box=(x0,y0,x1,y1)
    #save to labels file and dir
#   croped_image= crop_img(image,box)
    bb[image] = box

In [ ]:
#define accuracy metric
def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

# data generator 

In [ ]:
# expnd boxes to compensate for bounding box errors
crop_margin = 0.1
def expand_bb(image,box,margin=crop_margin):
    size_x,size_y=image.shape[1],image.shape[0]
    x0, y0, x1, y1 = box[0],box[1],box[2],box[3]
    dx = x1 - x0
    dy = y1 - y0
    x0 = max(0,x0-dx * crop_margin)
    x1 = min(size_x,x1+ dx * crop_margin + 1)
    y0 = max(0,y0-dy * crop_margin)
    y1 = min(size_y, y1+dy * crop_margin + 1)
    return x0,y0,x1,y1

def crop_img(image,box):
    new_box=expand_bb(image,box)
    if len(image.shape)==3:
        new_image = image[int(new_box[1]):int(new_box[3]), int(new_box[0]):int(new_box[2]),:]
    else:
        new_image = image[int(new_box[1]):int(new_box[3]), int(new_box[0]):int(new_box[2])]
    return new_image

In [ ]:
RESIZE_IMG = 299
import tensorflow as tf
from skimage.transform import resize
from keras.utils import np_utils
import cv2
from PIL import Image

def read_crop_resize(image_path,boxes=bb):
    image = cv2.imread(image_path)
    if image is not None:
        if image_path.split("/")[-1] in boxes:
            box=boxes[image_path.split("/")[-1]]
            croped_image= crop_img(image,box)
            image=croped_image
        try:
            image = cv2.resize(image, (RESIZE_IMG, RESIZE_IMG)) 
        except cv2.error as e:
            print("error resizing image:"+image_path )
            return None
    return image

def preprocess_images(image_names, seed, datagen,directory=TRAIN_IMG):
    np.random.seed(seed)
    X = np.zeros((len(image_names), RESIZE_IMG, RESIZE_IMG,3))
    for i, image_name in enumerate(image_names):
        if os.path.isfile(image_name):
            image = read_crop_resize(image_name)
        else:
            image = read_crop_resize(os.path.join(directory, image_name))
        if image is not None:
            if datagen is not None:
                image = datagen.random_transform(image)
            else:
                image=image
            image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image=np.expand_dims(image, axis=2)
            X[i]=np.repeat(image, 3, 2)
#             X[i]=tf.image.grayscale_to_rgb(image)
    return X

def image_generator(data_xy,batch_size,directory,augment=True,shuffle=True):
    datagen_args = dict(rescale=1./255,rotation_range=10,shear_range=0.2,
                        horizontal_flip=True)
    if not augment:
        datagen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**datagen_args)
    
    while True:
        # loop once per epoch
        num_recs = len(data_xy)
        if shuffle:
            indices = np.random.permutation(np.arange(num_recs))
        else:
            indices = np.arange(num_recs)
        num_batches = num_recs // batch_size
        for bid in range(num_batches):
            # loop once per batch
            batch_indices = indices[bid * batch_size : (bid + 1) * batch_size]
            batch = [data_xy[i] for i in batch_indices]
            # make sure image data generators generate same transformations
            seed = np.random.randint(low=0, high=1000, size=1)[0]
            X = preprocess_images([b[0] for b in batch], seed, 
                                      datagen, directory)
            
            
            Y = np.array([b[1] for b in batch]) # 0 or 1
            Y= np_utils.to_categorical(Y, num_classes=2932)
            yield (X.astype(np.uint8), Y)

In [ ]:
# damages/bad images to exclude. partially taken from https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563/data
images=['859e1399e.jpg'
#         ,'0b1e39ff.jpg','0c11fa0c.jpg','1b089ea6.jpg','2a2ecd4b.jpg','2c824757.jpg',
#         '3e550c8a.jpg','56893b19.jpg','613539b4.jpg','6530809b.jpg','6b753246.jpg',
#         '6b9f5632.jpg','75c94986.jpg','7f048f21.jpg','7f7702dc.jpg','806cf583.jpg',
#         '95226283.jpg','a3e9070d.jpg','ade8176b.jpg','b1cfda8a.jpg','b24c8170.jpg',
#         'b7ea8be4.jpg','b9315c19.jpg','b985ae1e.jpg','baf56258.jpg','c4ad67d8.jpg',
#         'c5da34e7.jpg','c5e3df74.jpg','ced4a25c.jpg','d14f0126.jpg','e0b00a14.jpg',
#         'e6ce415f.jpg','e9bd2e9c.jpg','f4063698.jpg','f9ba7040.jpg'
       ]
bad_images= set(images)
# 'a' in a_set

In [ ]:
encoder = LabelEncoder()
train_lbl["transformed_label"] = encoder.fit_transform(train_lbl["Id"])
# print(trafomed_label)

traindata=[]
for i in range(len(train_lbl)):
    x=train_lbl["Image"].iloc[i]
    y=train_lbl["transformed_label"].iloc[i]
#     if x in bad_images: continue
    traindata.append((x,y))

In [ ]:
testdata=[]
for i in range(len(test_df)):
    x=test_df["Image"].iloc[i]
    y=0
#     if x in bad_images: continue
    testdata.append((x,y))

In [ ]:
batch_gen = image_generator(traindata, 32,TRAIN_IMG)
(X, Y) = batch_gen.__next__()
print("generator output shapes:")
print(X.shape, Y.shape)

In [ ]:
plt.imshow(X[15])
# plt.imshow(X[0][:,:,0])

In [ ]:
from sklearn.model_selection import train_test_split
train,validation =train_test_split(traindata, test_size=0.2, random_state=42)

In [ ]:
train_generator = image_generator(train, 32,directory=TRAIN_IMG)
validation_generator = image_generator(validation, 32,directory=TRAIN_IMG)

In [ ]:
STEP_SIZE_TRAIN=0.8*len(traindata)//32
STEP_SIZE_VALID=0.2*len(traindata)//32

In [ ]:
CLASSES = len(np.unique(train_lbl['transformed_label'])) 
print("number of unique classes:"+ str(CLASSES))

**Transfer learning with Inception-resnet-v2**

In [ ]:
def model_transfer(image_size=RESIZE_IMG):
    checkpointer = ModelCheckpoint(filepath='transfetlr_mid_bb.hdf5', 
                               verbose=1, save_best_only=True)
    decayedlr= ReduceLROnPlateau(monitor='val_loss',patience=15,min_lr=1e-9,verbose=1,mode='min')
    
    # setup model
    input_tensor = Input(shape=(image_size,image_size, 3))
#     base_model = Xception(weights='imagenet', include_top=False,input_shape = (image_size,image_size, 3))
    base_model = InceptionResNetV2(weights='imagenet', include_top=False,input_shape = (image_size,image_size, 3)) 

    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(0.4)(x)
    predictions = Dense(CLASSES+1, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=predictions)

    layer_num = len(model.layers)
    for layer in model.layers[:int(layer_num * 0.9)]:
        layer.trainable = False
    
    for layer in model.layers[int(layer_num * 0.9):]:
        layer.trainable = True

    model.compile(optimizer='adagrad',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', top_5_accuracy])
    
    return model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from PIL import Image as pil_image
transfer_model=model_transfer()
plot_model(transfer_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=True)
pil_image.open('model_plot.png')

In [ ]:
if not LOAD_MODEL:
    start = time.time()
    print("start time:"+str(start))

    transfer_model.save('inceptionresnetv2_model_bb_last.h5')
    
    history = transfer_model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=validation_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=40, callbacks = [checkpointer,decayedlr])
    transfer_model.save('inceptionresnetv2_model_bb_last.h5')

    end = time.time()
    print("end time:"+str(end))
    print("Complete time: " + str(end - start) + " Secs.")

# Performance

In [ ]:
if not LOAD_MODEL:
    # if TRAIN:
        # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    history.params

In [ ]:
if not LOAD_MODEL:
    # if TRAIN:
        # summarize history for accuracy
    plt.plot(history.history['top_5_accuracy'])
    plt.plot(history.history['val_top_5_accuracy'])
    plt.title('model top 5 accuracy')
    plt.ylabel('top 5 accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    history.params

In [ ]:
if not LOAD_MODEL:
        # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Submission

In [ ]:
SUBMIT=True

In [ ]:
if SUBMIT:
    if LOAD_MODEL:
        if os.path.isfile(PRETRAIN_MODEL):
            model=load_model(PRETRAIN_MODEL,custom_objects={"top_5_accuracy": top_5_accuracy})
        if os.path.isfile(PRETRAIN_WEIGHTS):
            model.load_weights(PRETRAIN_WEIGHTS)
    else:
       model= transfer_model

In [ ]:
#submission
if SUBMIT:
    test_generator= image_generator(testdata, 1,directory=TEST_IMG,augment=False,shuffle=False)
    STEP_SIZE_TEST=len(testdata)
    pred = model.predict_generator(test_generator,verbose = 1,steps=STEP_SIZE_TEST,workers=1)
    pred_sorted = np.argsort(-pred, axis = 1)[:,:5]
    pred_sorted_big = np.argsort(-pred, axis = 1)[:,:100]
    top_pred=np.sort(-pred, axis = 1)[:,:1]
    top_5_pred=np.sort(-pred, axis = 1)[:,:5]
    # pred_sorted

In [ ]:
if SUBMIT:
    from tqdm import tqdm
    #create empty list
    pred_ids = list()
    test_imgs=list()
    for i,row in enumerate(tqdm(pred_sorted)):
        test_imgs.append(testdata[i][0])
        preds_encoded=row
        whales=encoder.inverse_transform(preds_encoded)
        pred_ids.append(whales)

In [ ]:
if SUBMIT:
    ### for pred big file
    from tqdm import tqdm
    #create empty list
    pred_ids_big = list()
    for i,row in enumerate(tqdm(pred_sorted_big)):
        preds_encoded=row
        whales=encoder.inverse_transform(preds_encoded)
        pred_ids_big.append(whales)

In [ ]:
if SUBMIT:
    final_preds = []
    for i,top_5_ids in enumerate(pred_ids):
        final_preds.append(' '.join(pred_ids[i]))
        
    import shutil
    shutil.rmtree('test_folder', ignore_errors=True)
    
    submission = pd.DataFrame({"Image": test_imgs, "Id": final_preds})
    submission.to_csv("submission.csv", index = False) #disabled to not override current submission
    submission.head()

In [ ]:
if SUBMIT:
    ### for pred big file (top100)
    final_preds = []
    
    for i,top_5_ids in enumerate(pred_ids_big):
        final_preds.append(' '.join(pred_ids_big[i]))
        
    import shutil
    shutil.rmtree('test_folder', ignore_errors=True)
    
    top_pred=-1*top_pred
    top_5_pred=-1*top_5_pred
    submission = pd.DataFrame({"Image": test_imgs,"top_score":list(top_pred), "Id": final_preds})
    submission.to_csv("submission_top100.csv", index = False)
    submission.head(50)
    
    submission_scores=  pd.DataFrame({"Image": test_imgs,"1_score":list(top_5_pred[:,0]),"2_score":list(top_5_pred[:,1]),"3_score":list(top_5_pred[:,2]),"4_score":list(top_5_pred[:,3]),"5_score":list(top_5_pred[:,4])})
    submission.to_csv("submission_scores_top5.csv", index = False)
    